# Classifying Text Values

In [181]:
#import additional sklearn functions
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from time import time
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

1. Import data and set X & y

In [147]:
#bring in test data for model to predict
path_to_file = "C:\\Users\\maweb\\Documents\\ThesisCode\\appeals_query.csv"
appeals_data = pd.read_csv(path_to_file, encoding='latin-1')

In [148]:
#assign the x_train data
X = appeals_data['Judgment'].head(100)
y = appeals_data['Denied']

In [149]:
X.iloc[0]

"Applicant's drug abuse was not mitigated where marijuana use was recent, and had continued after Applicant stated an intent to refrain from drug use in the future. He falsified his drug abuse history on security questionnaires in March and October 1995 an"

In [150]:
y.iloc[0]

True

2. Set up bag of words for Judgment field

In [151]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(X.tolist())


In [152]:
vectorizer.get_feature_names()

['00',
 '000',
 '10',
 '1001',
 '12',
 '13',
 '14',
 '15',
 '18',
 '19',
 '1967',
 '1972',
 '1976',
 '1977',
 '1978',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '20',
 '23',
 '25',
 '26',
 '30',
 '33',
 '35',
 '45',
 '60',
 '80',
 '86',
 '91',
 '94',
 '96',
 'aa',
 'ab',
 'abandonment',
 'able',
 'absence',
 'abstain',
 'abstinence',
 'abstinent',
 'abuse',
 'abused',
 'abuser',
 'abusing',
 'abusive',
 'access',
 'accident',
 'accounts',
 'action',
 'actions',
 'actively',
 'activity',
 'acts',
 'addicted',
 'addiction',
 'addictions',
 'addition',
 'adjudicative',
 'administrative',
 'adverse',
 'affirmed',
 'aftercare',
 'age',
 'ago',
 'alc',
 'alcohol',
 'alcoholism',
 'alford',
 'amd',
 'analysis',
 'anecdotal',
 'annual',
 'annually',
 'ap',
 'appeal',
 'appealing',
 'appeared',
 'applic',
 'applicant',
 'applicants',
 'application',
 'applications',
 'appllicant',
 'apply',
 'a

In [153]:
X[:1]

<1x758 sparse matrix of type '<class 'numpy.float64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [154]:
X.shape

(100, 758)

3. Change True values to 1.0 and False values to 0.0

In [165]:
y = y.replace(True,1).head(100)

In [166]:
y.value_counts()

1.0    65
0.0    35
Name: Denied, dtype: int64

In [167]:
X.shape

(100, 758)

In [168]:
y.shape

(100,)

4. Run train_test_split on the current X & y

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [169]:
X_train.shape

(67, 758)

In [170]:
X_test.shape

(33, 758)

5. Run classifiers on current data

In [182]:
classifiers = [RandomForestClassifier(n_estimators=5), GaussianNB(), LogisticRegression(solver='liblinear'), DecisionTreeClassifier(criterion='gini'), KNeighborsClassifier(n_neighbors=6)]
clf_names = ['RandomForest','GausianNB','LogisticRegression','DecisionTreeClassRegressor', 'KNeighbors']
metric_names = ['roc_auc','f1','accuracy','precision','recall']

scv = StratifiedKFold(n_splits=3)

scores_df = pd.DataFrame(index=metric_names,columns=clf_names)
clf_scores = []
for clf, name in zip(classifiers, clf_names):
    print('-------------------------------------------------------------------')
    print('clf: ',clf)
    print('')
    for metric in metric_names:
        score = cross_val_score(clf,X.toarray(),y,scoring=metric, cv=scv).mean()
        clf_scores.append(score)
        print('\t*',metric,' score: ', score)
    scores_df[name] = clf_scores
    clf_scores = []


-------------------------------------------------------------------
clf:  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

	* roc_auc  score:  0.7586580086580087
	* f1  score:  0.8717228286663503
	* accuracy  score:  0.7800245098039215
	* precision  score:  0.7385432667690733
	* recall  score:  0.9372294372294373
-------------------------------------------------------------------
clf:  GaussianNB(priors=None, var_smoothing=1e-09)

	* roc_auc  score:  0.7236652236652237
	* f1  score:  0.842962962962963
	* accuracy  score:  0.7787990196078431
	* precision  score:  0.7898550724637681
	* recall  score:  0.906926406926407
